In [1]:
import findspark
findspark.init()

In [10]:
from pyspark import SparkContext
sc = SparkContext("local", "WordCountCache")

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=BroadCast, master=local) created by __init__ at /tmp/ipykernel_12106/4025166763.py:2 

In [3]:
sector_dict = {
        "MSFT": "TECH",
        "TSLA": "AUTO",
        "EMR": "INDUSTRIAL"
}

stocks = [
    ("EMR", 52.0),
    ("TSLA", 300.0),
    ("MSFT", 100.0)
]

In [4]:
# create broadcasted variabel using Spark Context
# this will ensure that sector_dict is kept in every executor 
# where task shall be running
# lazy evaluation, data shall be copied to executors when we run the job
broadCastSectorDict = sc.broadcast(sector_dict)

In [5]:
stocksRdd = sc.parallelize(stocks)


In [6]:
# Pyspark see this code, this has reference to broadCastSectorDict
# which is broardcast data, pyspark place the broadCastSectorDict in every executor
# 1 time instead of every job
# without broadCast, sector_dict shall be copied to executor for every task
# add sector code with stock at executor level when task running
def enrichStockWithSector(stock):
    return stock + (broadCastSectorDict.value[stock[0]] ,)

In [7]:
# code marshalling - Python copy the lamnda code to executor system/processor
# now enrichStockWithSector also shipped to executor on every task
enrichedRdd = stocksRdd.map (lambda stock: enrichStockWithSector(stock))

In [8]:
enrichedRdd.collect()


[('EMR', 52.0, 'INDUSTRIAL'), ('TSLA', 300.0, 'AUTO'), ('MSFT', 100.0, 'TECH')]

In [9]:
enrichedRdd.take(5)

[('EMR', 52.0, 'INDUSTRIAL'), ('TSLA', 300.0, 'AUTO'), ('MSFT', 100.0, 'TECH')]